<a href="https://colab.research.google.com/github/antonum/Timescale-Workshops/blob/main/Terraform_Timescale_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Provision Timescale service with Terraform

### References

- Timescale terraform provider https://registry.terraform.io/providers/timescale/timescale/latest/docs
- Timescale documentation https://docs.timescale.com/use-timescale/latest/integrations/terraform/

### Prerequisites

You should have an existing project in the Terraform cloud. You can create a free 30 days trial (no credit card required) here: https://console.cloud.timescale.com/

In [2]:
%%bash
# install terraform
wget -O - https://apt.releases.hashicorp.com/gpg | sudo gpg --dearmor -o /usr/share/keyrings/hashicorp-archive-keyring.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/hashicorp.list
sudo apt update && sudo apt install terraform -q

deb [arch=amd64 signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com jammy main
Get:1 https://apt.releases.hashicorp.com jammy InRelease [12.9 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://apt.releases.hashicorp.com jammy/main amd64 Packages [217 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,383 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get

--2025-04-10 00:28:19--  https://apt.releases.hashicorp.com/gpg
Resolving apt.releases.hashicorp.com (apt.releases.hashicorp.com)... 18.154.185.73, 18.154.185.31, 18.154.185.57, ...
Connecting to apt.releases.hashicorp.com (apt.releases.hashicorp.com)|18.154.185.73|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3980 (3.9K) [binary/octet-stream]
Saving to: ‘STDOUT’

     0K ...                                                   100% 1.21G=0s

2025-04-10 00:28:19 (1.21 GB/s) - written to stdout [3980/3980]



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: una

## Create .tf files

Following [Terraform best practices](https://www.terraform-best-practices.com/code-structure#getting-started-with-the-structuring-of-terraform-configurations) we'll structure code as `main.tf`, `variables.tf` and `outputs.tf` and `terraform.tfvars`. The code below creates these files in the home directory of this notebook.

Instead of putting your credentials in `terraform.tfvars` you can use `-var` argment in terraform command line or environment variables:

```
export TF_VAR_ts_access_key=01JR8823FVNHNQZY0TJBXXXXX
export TF_VAR_ts_secret_key=j68Od78cDOKJItUCnWoC7PQPkGSkIpbPaaU2aquwkErrqIYgB3xCmAxxxXXXxx
export TF_VAR_ts_project_id=ocssxxxx
```


In [22]:
%%bash
# Create terraform.tfvars
tee terraform.tfvars <<'EOF'
# use keys and project ID from your existing project

ts_access_key="01JR8823FVNHNQZY0TJBXXXX"
ts_secret_key="j68Od78cDOKJItUCnWoC7PQPkGSkIpbPaaU2aquwkErrqIYgB3xCmAQXV71xxxxx"
ts_project_id="ocssgixxxx"

EOF

# use keys and project ID from your existing project

ts_access_key="01JR8823FVNHNQZY0TJBXXXX"
ts_secret_key="j68Od78cDOKJItUCnWoC7PQPkGSkIpbPaaU2aquwkErrqIYgB3xCmAQXV71xxxxx"
ts_project_id="ocssgixxxx"



In [4]:
%%bash
# Create main.tf
tee main.tf <<EOF
terraform {
  required_providers {
    timescale = {
      source  = "timescale/timescale"
      version = "~> 1.13.1"
    }
  }
}

# Authenticate using client credentials.
# They are issued through the Timescale UI.
# When required, they will exchange for a short-lived JWT to do the calls.
provider "timescale" {
  project_id = var.ts_project_id
  access_key = var.ts_access_key
  secret_key = var.ts_secret_key
}

resource "timescale_service" "test" {
  name       = "anton-tf-test"
  milli_cpu  = 500
  memory_gb  = 2
  region_code = "us-east-1"
  enable_ha_replica = false
  timeouts = {
    create = "30m"
  }
}
EOF



terraform {
  required_providers {
    timescale = {
      source  = "timescale/timescale"
      version = "~> 1.13.1"
    }
  }
}

# Authenticate using client credentials.
# They are issued through the Timescale UI.
# When required, they will exchange for a short-lived JWT to do the calls.
provider "timescale" {
  project_id = var.ts_project_id
  access_key = var.ts_access_key
  secret_key = var.ts_secret_key
}

resource "timescale_service" "test" {
  name       = "anton-tf-test"
  milli_cpu  = 500
  memory_gb  = 2
  region_code = "us-east-1"
  enable_ha_replica = false
  timeouts = {
    create = "30m"
  }
}


In [5]:
%%bash
# Create variables.tf
tee variables.tf <<EOF
variable "ts_project_id" {
  type = string
  //default = "ocssgijxxx"
}

variable "ts_access_key" {
  type = string
  //default = "01JR8823FVNHNQZY0TJB6XXXXX"
}

variable "ts_secret_key" {
  type      = string
  sensitive = true
  //default = "j68Od78cDOKJItUCnWoC7PQPkGSkIpbPaaU2aquwkErrqIYgB3xCmxxXXxxXX"
}
EOF

variable "ts_project_id" {
  type = string
  //default = "ocssgijxxx"
}

variable "ts_access_key" {
  type = string
  //default = "01JR8823FVNHNQZY0TJB6XXXXX"
}

variable "ts_secret_key" {
  type      = string
  sensitive = true
  //default = "j68Od78cDOKJItUCnWoC7PQPkGSkIpbPaaU2aquwkErrqIYgB3xCmxxXXxxXX"
}


In [6]:
%%bash
# Create outputs.tf
tee outputs.tf <<'EOF'
output "timescale_service_hostname" {
  value = timescale_service.test.hostname
}

output "timescale_service_port" {
  value = timescale_service.test.port
}

output "timescale_service_password" {
  value = timescale_service.test.password
  sensitive = true
}

output "timescale_service_psql" {
  value = "psql -d \"postgres://tsdbadmin:${timescale_service.test.password}@${timescale_service.test.hostname}:${timescale_service.test.port}/tsdb?sslmode=require\""
  sensitive = true
}
EOF

output "timescale_service_hostname" {
  value = timescale_service.test.hostname
}

output "timescale_service_port" {
  value = timescale_service.test.port
}

output "timescale_service_password" {
  value = timescale_service.test.password
  sensitive = true
}

output "timescale_service_psql" {
  value = "psql -d \"postgres://tsdbadmin:${timescale_service.test.password}@${timescale_service.test.hostname}:${timescale_service.test.port}/tsdb?sslmode=require\""
  sensitive = true
}


In [7]:
%%bash
terraform init

Initializing the backend...
Initializing provider plugins...
- Finding timescale/timescale versions matching "~> 1.13.1"...
- Installing timescale/timescale v1.13.1...
- Installed timescale/timescale v1.13.1 (signed by a HashiCorp partner, key ID B56785AF7539DE30)
Partner and community providers are signed by their developers.
If you'd like to know more about provider signing, you can read about it here:
https://developer.hashicorp.com/terraform/cli/plugins/signing
Terraform has created a lock file .terraform.lock.hcl to record the provider
selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or 

In [8]:
%%bash
terraform validate

Success! The configuration is valid.



In [18]:
%%bash
terraform plan


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # timescale_service.test will be created
  + resource "timescale_service" "test" {
      + connection_pooler_enabled = false
      + enable_ha_replica         = false
      + environment_tag           = (known after apply)
      + hostname                  = (known after apply)
      + id                        = (known after apply)
      + memory_gb                 = 2
      + milli_cpu                 = 500
      + name                      = "anton-tf-test"
      + password                  = (sensitive value)
      + paused                    = false
      + pooler_hostname           = (known after apply)
      + pooler_port               = (known after apply)
      + port                      = (known after apply)
      + region_code               = "us-east-1"
      + replica_hostna

## Provision Timescale Cloud service

In the cell below enter access key, secret key and project ID of your timescale cloud project. To get the key navigate to Project (upper left corner) -> Project Settings -> Client Credentials -> Create Credentials

**WARNING!!!** provisioning new service on Timescale Cloud might incur additional costs. Make sure to delete the service. Cleanup cell below.

In [19]:
%%bash
terraform apply  -auto-approve


Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # timescale_service.test will be created
  + resource "timescale_service" "test" {
      + connection_pooler_enabled = false
      + enable_ha_replica         = false
      + environment_tag           = (known after apply)
      + hostname                  = (known after apply)
      + id                        = (known after apply)
      + memory_gb                 = 2
      + milli_cpu                 = 500
      + name                      = "anton-tf-test"
      + password                  = (sensitive value)
      + paused                    = false
      + pooler_hostname           = (known after apply)
      + pooler_port               = (known after apply)
      + port                      = (known after apply)
      + region_code               = "us-east-1"
      + replica_hostna

In [ ]:
# output psql connection string for your service
!terraform output timescale_service_psql

## Cleanup

In [21]:
%%bash
# delete service
terraform destroy  -auto-approve

timescale_service.test: Refreshing state... [id=sjm7ygzz7c]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # timescale_service.test will be destroyed
  - resource "timescale_service" "test" {
      - connection_pooler_enabled = false -> null
      - enable_ha_replica         = false -> null
      - environment_tag           = "DEV" -> null
      - hostname                  = "sjm7ygzz7c.ocssgijfrc.tsdb.cloud.timescale.com" -> null
      - id                        = "sjm7ygzz7c" -> null
      - memory_gb                 = 2 -> null
      - milli_cpu                 = 500 -> null
      - name                      = "anton-tf-test" -> null
      - password                  = (sensitive value) -> null
      - paused                    = false -> null
      - port                      = 37218 -> null
      - region_code               = "